Load libraries.  In this case we need the ndex library, which you can get with "pip install ndex"

In [1]:
import ndex2

In [2]:
import requests
import json
import base64

# Connect to NDEx in order to retrieve a network.

Via `ndex2.create_nice_cx_from_server()`, we grab the [NCI Pathway Interaction Database - Final Revision - Extended Binary SIF](http://ndexbio.org/#/network/c0e70804-d848-11e6-86b1-0ac135e8bacf) network from NDEx, using its UUID.  

The network is downloaded to create a [NiceCX](https://ndex2.readthedocs.io/en/latest/ndex2.html#nicecxnetwork) object `nice_cx_net`. 

In [3]:
nice_cx_net = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org',
                                               uuid='c0e70804-d848-11e6-86b1-0ac135e8bacf')

# Select seed nodes for diffusion

To set seed nodes, add the node attribute `diffusion_input` with a value of `1.0`

The code below grabs the first node, setting the `diffusion_input` node attribute. 


In [4]:

for nodeid, nodeobj in nice_cx_net.get_nodes():
    nice_cx_net.remove_node_attribute(node=nodeid, attribute_name='diffusion_input')
    nice_cx_net.set_node_attribute(type='double', node=nodeid, attribute_name='diffusion_input', values='1.0')
    print(nice_cx_net.get_node_attribute(nodeid,'diffusion_input'))
    # break out of loop so we only set the attribute on the first node
    break

{'po': 3431, 'n': 'diffusion_input', 'v': '1.0', 'd': 'double'}


# Call the Diffusion service

Here the diffusion service is called. This can take a few minutes to run. 

#### NOTE: Diffusion uses the CXmate service, which requires all attributes to be passed as strings. So we convert the network to CX and stringify attributes

In [5]:
url = 'http://v3.heat-diffusion.cytoscape.io'
payload = nice_cx_net.to_cx()
for p in payload:
    k = list(p.keys())[0]
    if 'Attributes' in k:
        for i in range(len(p[k])):
            p[k][i]['v'] = str(p[k][i]['v'])
response = requests.post(url, json=payload)

Generating CX


# Add results from diffusion to our network

The `data` dictionary in the json response `response.json()['data']` is a dictionary that contains [CX](https://home.ndexbio.org/data-model/) formatted data with results of diffussion. This next fragment of code grabs the results of diffusion

In [6]:
for aspect in response.json()['data']:
    if 'nodeAttributes' not in aspect:
        continue
    for n_attr in aspect['nodeAttributes']:
        if n_attr['n'] == 'diffusion_output_rank' or n_attr['n'] == 'diffusion_output_heat':
            if n_attr['d'] == 'float':
                n_type = 'double'
            else:
                n_type = n_attr['d']
            nice_cx_net.add_node_attribute(property_of=int(n_attr['po']),
                                 name=n_attr['n'],
                                 values=n_attr['v'],
                                 type=n_type)



# Output best results from diffusion

Here the code iterates through the nodes of the network looking for the 
`10` best hits (by examining `diffusion_output_rank`).

In [7]:
nos = []
for node_id, node in nice_cx_net.get_nodes():
    rank = nice_cx_net.get_node_attribute_value(node_id, 'diffusion_output_rank')
    if int(rank) < 10:
        node['diffusion_output_rank'] = rank
        heat = nice_cx_net.get_node_attribute_value(node_id, 'diffusion_output_heat')
        node['diffusion_output_heat'] = float(heat)
        nos.append(node)

# sort the result
nos = sorted(nos, key=lambda k: k['diffusion_output_rank'])       
nos

[{'@id': 2878,
  'n': 'BRAF',
  'diffusion_output_rank': '0',
  'diffusion_output_heat': 0.06310134724929448},
 {'@id': 2600,
  'n': 'PAK4',
  'diffusion_output_rank': '1',
  'diffusion_output_heat': 0.03045349811890815},
 {'@id': 2147,
  'n': 'PPP2R2B',
  'diffusion_output_rank': '2',
  'diffusion_output_heat': 0.030320880954123476},
 {'@id': 2397,
  'n': 'KSR1',
  'diffusion_output_rank': '3',
  'diffusion_output_heat': 0.025078867225448172},
 {'@id': 1340,
  'n': 'RIT2',
  'diffusion_output_rank': '4',
  'diffusion_output_heat': 0.022151178863522168},
 {'@id': 1337,
  'n': 'RIT1',
  'diffusion_output_rank': '5',
  'diffusion_output_heat': 0.022151178863522168},
 {'@id': 1857,
  'n': 'PPP2R1A',
  'diffusion_output_rank': '6',
  'diffusion_output_heat': 0.02035448507221378},
 {'@id': 1251,
  'n': 'RANBP10',
  'diffusion_output_rank': '7',
  'diffusion_output_heat': 0.019835930214747586},
 {'@id': 1431,
  'n': 'MUC20',
  'diffusion_output_rank': '8',
  'diffusion_output_heat': 0.018817

# Demo completed. Have a nice day!